In [ ]:
# --- Imports Essenciais ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from sklearn.linear_model import LinearRegression
from IPython.display import HTML # Para exibir a animação no notebook


# --- Configurações de Estilo e Avisos ---
import warnings
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid') # Deixa os gráficos mais bonitos

print("Bibliotecas importadas com sucesso.")

# --- Definição das Bandas Contratadas (em bps) ---
BANDAS_ESPECIFICAS = {
    'dc:a6:32:6b:9a:da': 500_000_000, 
    'dc:a6:32:6b:9c:a8': 600_000_000,
    'e4:5f:01:ad:56:31': 350_000_000,
    'e4:5f:01:36:10:3e': 1_000_000_000,
    'e4:5f:01:8e:52:7a': 1_000_000_000,
    'e4:5f:01:b4:bb:d4': 750_000_000,
    '80:af:ca:27:f3:0e': 1_000_000_000,
    '98:25:4a:b9:46:23': 1_000_000_000
}
BANDA_PADRAO = 100_000_000 # Valor padrão se um MAC não estiver na lista

In [ ]:
# Importa o CSV e transforma em DataFrame
df = pd.read_csv('ndt_tests_att.csv')

# Agrupar dados por pares de MAC address e server IP
pares_unicos = df.groupby(['mac_address', 'server_ip']).size().reset_index(name='count')
print(f"Total de pares únicos: {len(pares_unicos)}")
print(f"Total de registros: {pares_unicos['count'].sum()}")
# Exibir estatísticas por par
print("\nEstatísticas por par:")

# Ordena os pares por número de registros (do maior para o menor)
pares_ordenados = pares_unicos.sort_values('count', ascending=False)
pares_ordenados = pares_ordenados[pares_ordenados['count'] >= 70]
print(len(pares_ordenados), "pares com pelo menos 70 registros.")
# Filtrar o DataFrame para manter apenas os pares selecionados
pares_selecionados = pares_ordenados[['mac_address', 'server_ip']]
df = df.merge(pares_selecionados, on=['mac_address', 'server_ip'], how='inner')
df = df.drop(columns=['download_retrans_percent', 'test_uuid', 'client_ip'])

# Filtrando timestamps
# Converte e filtra o timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df = df[df['timestamp'] >= '2025-05-28']
TIMESTAMPS_PARA_REMOVER = [
    '2025-06-18 15:29:04',
    '2025-07-13 01:09:40',
    '2025-06-09 21:08:31',
    '2025-06-08 13:12:10',
    '2025-06-30 19:47:38',
    '2025-07-09 00:08:06',
    '2025-06-20 21:29:28',
    '2025-07-06 12:50:32',
    '2025-06-25 03:59:45',
    '2025-06-16 03:22:18',
    '2025-07-08 19:51:17',
    '2025-07-10 01:15:49',
    '2025-06-16 09:15:57',
    '2025-06-30 00:31:50',
    '2025-06-08 15:12:18',
    '2025-05-29 04:13:34',
    '2025-05-29 09:04:43',
    '2025-05-29 13:21:28',
    '2025-05-30 04:14:04',
    '2025-07-03 20:50:39',
    '2025-05-28 17:36:20',
    '2025-05-28 18:22:46',
    '2025-05-28 18:52:46',
    '2025-05-28 20:52:46',
    '2025-05-28 22:21:29',
    '2025-05-29 00:32:56',
    '2025-05-29 01:43:34',
    '2025-05-29 03:43:34',
    '2025-05-29 04:43:34',
    '2025-05-29 05:55:28',
    '2025-05-29 07:23:55',
    '2025-05-29 07:56:51',
    '2025-05-29 10:52:49',
    '2025-05-29 11:31:45',
    '2025-05-29 12:13:43',
    '2025-05-29 14:26:56',
    '2025-05-29 16:55:14',
    '2025-05-29 18:07:22',
    '2025-05-29 20:37:22',
    '2025-05-29 22:44:27',
    '2025-05-29 23:59:48',
    '2025-05-30 02:14:03',
    '2025-05-30 04:44:04',
    '2025-05-30 05:55:06',
    '2025-05-30 06:51:39',
    '2025-05-30 08:01:50',
    '2025-05-30 09:51:51',
    '2025-05-30 11:20:22',
    '2025-05-30 12:55:19',
    '2025-05-30 13:25:18',
    '2025-05-30 14:14:04',
    '2025-05-30 15:25:18',
    '2025-05-30 16:47:34',
    '2025-05-30 17:36:20',
    '2025-05-30 18:22:46',
    '2025-05-30 19:52:46',
    '2025-05-30 20:47:34',
    '2025-05-30 21:52:46',
    '2025-05-30 22:20:34',
    '2025-05-30 23:21:29',
    '2025-05-30 00:29:49',
    '2025-06-11 00:48:09',
    '2025-05-29 15:26:56',
    '2025-05-29 18:37:22',
    '2025-05-29 22:14:27',
    '2025-06-14 22:12:41',
    '2025-06-20 18:43:39',
    '2025-05-30 14:35:13',
    '2025-06-20 16:28:57',
    '2025-06-13 01:29:47',
    '2025-06-14 20:25:06',
    '2025-06-07 08:56:58',
    '2025-06-15 12:46:22',
    '2025-06-01 04:04:52',
    '2025-06-03 13:13:52',
    '2025-06-13 19:31:36',
    '2025-06-14 15:25:58',
    '2025-06-01 01:25:24',
    '2025-06-01 21:31:57',
    '2025-06-12 14:21:41',
    '2025-06-14 13:56:40',
    '2025-06-09 13:10:41',
    '2025-07-01 21:34:28',
    '2025-06-12 16:08:21',
    '2025-06-15 17:46:23',
    '2025-06-08 06:58:23',
    '2025-06-01 10:07:15',
    '2025-06-18 19:00:00',
    '2025-06-18 20:30:00',
    '2025-06-02 22:01:43',
    '2025-06-11 10:12:07',
    '2025-07-07 15:05:46',
    '2025-07-03 21:27:08',
    '2025-06-09 15:42:42',
    '2025-06-22 02:23:29',
    '2025-06-22 16:23:31',
    '2025-06-26 04:33:53',
    '2025-06-22 07:47:41',
    '2025-07-08 22:37:40',
    '2025-06-04 04:40:07',
    '2025-06-09 16:27:46',
    '2025-07-03 02:30:52',
    '2025-06-15 22:00:18',
    '2025-06-07 13:42:10',
    '2025-06-21 13:07:05',
    '2025-07-08 05:16:07',
    '2025-06-20 17:28:58',
    '2025-06-22 05:47:41',
    '2025-06-05 19:27:26',
    '2025-06-12 18:55:08',
    '2025-06-02 10:55:00',
    '2025-06-11 19:43:45',
    '2025-07-05 02:49:47',
    '2025-06-23 19:54:06',
    '2025-06-27 21:39:32',
    '2025-07-09 02:08:10',
    '2025-06-12 15:52:24',
    '2025-07-04 01:47:41',
    '2025-06-03 18:57:50',
    '2025-07-04 21:10:10',
    '2025-07-07 23:30:04',
    '2025-06-15 23:01:53',
    '2025-06-04 01:35:48'
]
timestamps_removidos = pd.to_datetime(TIMESTAMPS_PARA_REMOVER)
indices_para_remover = df[df['timestamp'].isin(timestamps_removidos)].index
df.drop(indices_para_remover, inplace=True) # Remove do DataFrame principal 'df'

# AJUSTE: Garante que os dados estão ordenados por tempo
df = df.sort_values(by='timestamp')

# Resetar o índice caso ele já exista de algum passo anterior
df.reset_index(drop=True, inplace=True)

print("DataFrame preparado e pronto para análise:")
print(df.head())
print(f"\nTotal de pontos após a filtragem: {len(df)}")

In [ ]:
# Criar uma figura com subplots para todos os pares analisados
fig, axes = plt.subplots(5, 5, figsize=(20, 16))
fig.suptitle('Dados de Download Throughput para Todos os Pares Analisados', fontsize=20, y=0.98)

# Achatar a matriz de eixos para iteração mais fácil
axes_flat = axes.flatten()

# Iterar pelos pares para análise
for idx, (_, linha_par) in enumerate(pares_para_analise.iterrows()):
    if idx >= 36:  # Limitamos a 36 pares (6x6 grid)
        break
        
    mac = linha_par['mac_address']
    server = linha_par['server_ip']
    
    # Filtrar dados para o par atual
    dados_par = df[(df['mac_address'] == mac) & (df['server_ip'] == server)]
    
    # Plotar no subplot correspondente
    ax = axes_flat[idx]
    ax.plot(dados_par['timestamp'], dados_par['download_tp_bps'] / 1e9, 'b.', alpha=0.7, markersize=5)
    
    # Formatação do subplot
    ax.set_title(f'{mac}→{server}', fontsize=8)
    ax.set_ylabel('Download (Gbps)', fontsize=8)
    ax.tick_params(axis='x', rotation=45, labelsize=6)
    ax.tick_params(axis='y', labelsize=6)
    ax.grid(True, alpha=0.3)

# Remover subplots vazios se houver menos de 25 pares
for idx in range(len(pares_para_analise), 25):
    axes_flat[idx].remove()

plt.tight_layout()
plt.subplots_adjust(top=0.94)
plt.show()

print(f"Plotados {min(len(pares_para_analise), 25)} pares de {len(pares_para_analise)} disponíveis")

In [ ]:
# ==================================================================
# ABORDAGEM 1: MODELO ARX (EQUAÇÃO DE DIFERENÇAS)
# ==================================================================

def estimar_coeficientes_arx(grupo_df):
    """
    Prepara os dados com colunas 'futuras' e estima os coeficientes do modelo ARX
    usando Regressão Linear.
    """
    # Define as 'causas' (features) e o 'efeito' (alvo)
    features = ['download_tp_bps', 'latency_download_sec', 'upload_tp_bps', 'latency_upload_sec']
    alvo = 'download_tp_bps' # Queremos prever o próximo download
    
    df_modelo = grupo_df[features].copy()
    
    # Cria a coluna alvo com o valor do próximo passo usando .shift(-1)
    df_modelo['alvo_futuro'] = df_modelo[alvo].shift(-1)
    
    # Remove a última linha que não tem um valor futuro
    df_modelo.dropna(inplace=True)
    
    if len(df_modelo) < 10: # A regressão precisa de alguns pontos para garantir uma boa estimativa
        return None

    # Prepara os dados para a regressão
    X = df_modelo[features]
    y = df_modelo['alvo_futuro']
    
    # Usamos a regressão linear da scikit-learn para encontrar os coeficientes
    reg = LinearRegression()
    reg.fit(X, y)
    
    # Retorna o intercepto (c0) e os outros coeficientes (c1, c2...)
    return {'intercepto': reg.intercept_, 'coeficientes': reg.coef_}

def simular_e_plotar_arx(par, dados_reais_par, resultado_arx):
    """
    Simula o modelo ARX passo a passo e plota o resultado.
    """
    print(f"\n--- Gerando Simulação ARX para o Par: {par} ---")
    
    # Pega os coeficientes encontrados
    intercepto = resultado_arx['intercepto']
    coeficientes = resultado_arx['coeficientes']
    
    # Usa a primeira linha de dados como ponto de partida da simulação
    estado_atual = dados_reais_par.iloc[0][['download_tp_bps', 'latency_download_sec', 'upload_tp_bps', 'latency_upload_sec']].to_numpy()
    previsoes = [estado_atual[0]] # Começa com o primeiro valor real de download
    
    # Loop de simulação: prevê um passo de cada vez
    for i in range(len(dados_reais_par) - 1):
        # A equação de diferenças: D[n+1] = c0 + c1*D[n] + c2*L_D[n] + ...
        previsao_proximo_d = intercepto + np.dot(estado_atual, coeficientes)
        previsoes.append(previsao_proximo_d)
        
        # Atualiza o estado para a próxima iteração
        # Usamos a previsão para o download e os valores reais para as outras variáveis
        estado_atual = dados_reais_par.iloc[i+1][['download_tp_bps', 'latency_download_sec', 'upload_tp_bps', 'latency_upload_sec']].to_numpy()
        estado_atual[0] = previsao_proximo_d

    # Plotagem
    plt.figure(figsize=(16, 8))
    plt.plot(dados_reais_par['timestamp'], dados_reais_par['download_tp_bps'], 'o', alpha=0.6, label='Download Real')
    plt.plot(dados_reais_par['timestamp'], previsoes, 'r.-', alpha=0.8, label='Simulação do Modelo ARX')
    plt.title(f"Validação do Modelo ARX para o Par {par}", fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Filtrar dados para o par específico
par_especifico = df[(df['mac_address'] == 'e4:5f:01:b4:bb:d4') & (df['server_ip'] == '200.159.254.239')]

# Plotar os dados reais
plt.figure(figsize=(16, 10))

# Subplot 1: Download throughput
plt.subplot(2, 2, 1)
plt.plot(par_especifico['timestamp'], par_especifico['download_tp_bps'], 'b.-', alpha=0.7)
plt.title('Download Throughput')
plt.xlabel('Timestamp')
plt.ylabel('Download (bps)')
plt.xticks(rotation=45)
plt.grid(True)

plt.suptitle(f"Dados Reais para MAC: dc:a6:32:6b:9a:da -> Server: 177.136.80.229\nTotal de pontos: {len(par_especifico)}", fontsize=16)
plt.tight_layout()
plt.show()

print(f"Estatísticas do par:")
print(f"Número de registros: {len(par_especifico)}")
print(f"Período: {par_especifico['timestamp'].min()} até {par_especifico['timestamp'].max()}")
# Encontrar o menor valor de download e seu timestamp
min_download = par_especifico['download_tp_bps'].min()
min_timestamp = par_especifico.loc[par_especifico['download_tp_bps'].idxmin(), 'timestamp']

print(f"Menor valor de download: {min_download:,.2f} bps")
print(f"Timestamp do menor valor: {min_timestamp}")

In [ ]:
# ==================================================================
# BLOCO DE EXECUÇÃO FINAL (CORRIGIDO)
# ==================================================================

print("Iniciando análise com o Modelo ARX para os pares selecionados...")

# Pega a lista de pares únicos que você já filtrou (com 70+ registros)
print(f"Total de pontos para análise: {len(df)}")
pares_para_analise = df.groupby(['mac_address', 'server_ip']).size().reset_index(name='count')
pares_para_analise = pares_para_analise[pares_para_analise['count'] >= 69]
print(f"Total de pares para análise: {len(pares_para_analise)}")
# --- AQUI ESTÁ O LOOP CORRIGIDO ---

# Usamos .iterrows() para iterar sobre as linhas do DataFrame de pares
for index, linha_par in pares_para_analise.iterrows():
    
    # Pega o mac e o server da linha atual
    mac = linha_par['mac_address']
    server = linha_par['server_ip']
    par = (mac, server)
    
    # Agora, filtra o DataFrame principal para pegar o grupo de dados deste par
    # Lembre-se de usar seu DataFrame limpo (df_limpo) aqui, se tiver um.
    grupo_df = df[(df['mac_address'] == mac) & (df['server_ip'] == server)]
    
    # 1. Estima os coeficientes para o par atual
    # A função 'estimar_coeficientes_arx' recebe o grupo de dados correto
    resultado_arx = estimar_coeficientes_arx(grupo_df)
    
    # 2. Se a estimação foi bem sucedida, simula e plota
    if resultado_arx:
        print(f"Coeficientes encontrados para {par}:")
        print(f"  Intercepto (c0): {resultado_arx['intercepto']:.2f}")
        print(f"  Coeficientes (c1..c4): {resultado_arx['coeficientes']}")
        simular_e_plotar_arx(par, grupo_df, resultado_arx)
    else:
        print(f"  ❌ Falha na estimação dos coeficientes para {par} (dados insuficientes)")

# Contador de gráficos gerados
if 'resultado_arx' in locals() and resultado_arx:
    graficos_gerados = sum(1 for _, linha in pares_para_analise.iterrows() 
                            if estimar_coeficientes_arx(df[(df['mac_address'] == linha['mac_address']) & 
                                                        (df['server_ip'] == linha['server_ip'])]) is not None)
    print(f"\n📊 Total de gráficos ARX gerados: {graficos_gerados}")
print("\nAnálise ARX finalizada.")

In [ ]:
# --- Definição da Classe para o Filtro RLS ---
class FiltroRLS:
    """
    Implementa um filtro Recursive Least Squares (RLS) para aprendizado online.
    """
    def __init__(self, num_variaveis, fator_esquecimento=0.99, P_inicial=1e6):
        self.num_vars = num_variaveis
        self.lam = fator_esquecimento
        self.theta = np.zeros(num_variaveis)
        self.P = P_inicial * np.identity(num_variaveis)

    def update(self, x, y):
        x = np.asarray(x).reshape(self.num_vars, 1)
        y = np.asarray(y)
        erro_previsao = y - (x.T @ self.theta)
        Px = self.P @ x
        ganho_k = Px / (self.lam + x.T @ Px)
        self.theta = self.theta + (ganho_k * erro_previsao).flatten()
        self.P = (1 / self.lam) * (self.P - (ganho_k @ x.T @ self.P))

    def predict(self, x):
        return np.dot(x, self.theta)



In [ ]:
def gerar_animacao_rls(df, mac_alvo, server_alvo, fator_esquecimento=0.995):
    """
    Gera e salva uma animação do processo de aprendizado RLS com escalas de gráfico aprimoradas.
    """
    print("="*80)
    print(f"Iniciando geração de animação para o par: {(mac_alvo, server_alvo)}")

    # 1. Preparação dos Dados
    dados_par = df[(df['mac_address'] == mac_alvo) & (df['server_ip'] == server_alvo)].sort_values(by='timestamp')
    if len(dados_par) < 20:
        print("--> DADOS INSUFICIENTES.")
        return None
    
    df_modelo = dados_par[['download_tp_bps', 'latency_download_sec', 'upload_tp_bps', 'latency_upload_sec']].copy()
    df_modelo['D_proximo'] = df_modelo['download_tp_bps'].shift(-1)
    df_modelo.dropna(inplace=True)
    df_modelo.insert(0, 'intercept', 1)
    
    X = df_modelo[['intercept', 'download_tp_bps', 'latency_download_sec', 'upload_tp_bps', 'latency_upload_sec']].to_numpy()
    y = df_modelo['D_proximo'].to_numpy()

    # 2. Inicialização do Filtro e Históricos
    num_coeficientes = X.shape[1]
    filtro = FiltroRLS(num_variaveis=num_coeficientes, fator_esquecimento=fator_esquecimento)
    historico_coeficientes = []
    historico_erro = []
    previsoes_online = []

    # 3. Configuração da Figura
    fig, axs = plt.subplots(2, 2, figsize=(18, 10), gridspec_kw={'height_ratios': [3, 2]})
    mac_formatado = mac_alvo.replace(':', '')
    # Depois, usamos essa variável já limpa na f-string
    fig.suptitle(f'Aprendizado Online (RLS) para {mac_formatado} -> {server_alvo}', fontsize=18)

    
    ax1 = axs[0, 0]; ax2 = axs[0, 1]; ax3 = axs[1, 0]; ax4 = axs[1, 1]
    ax1.plot(range(len(y)), y, 'o', color='skyblue', alpha=0.5, label='Download Real (Alvo)')
    line_pred, = ax1.plot([], [], 'r-', lw=2, label='Previsão do Modelo')
    ax1.set_title('Gráfico 1: Previsão vs. Real'); ax1.set_xlabel('Passo de Tempo'); ax1.set_ylabel('Download (bps)'); ax1.legend(); ax1.grid(True)
    ax1.set_xlim(0, len(y)) # Deixa o Y se ajustar dinamicamente

    line_erro, = ax2.plot([], [], '-', color='orange', lw=2)
    ax2.set_title('Gráfico 2: Erro da Previsão'); ax2.set_xlabel('Passo de Tempo'); ax2.set_ylabel('Erro (bps)'); ax2.grid(True); ax2.set_xlim(0, len(y)); ax2.axhline(0, color='black', lw=1, linestyle='--')
    
    labels_coeficientes = ['c0(Intercepto)', 'c1(D[n])', 'c2(L_D[n])', 'c3(U[n])', 'c4(L_U[n])']
    lines_coeffs = [ax3.plot([], [], lw=2, label=label)[0] for label in labels_coeficientes]
    ax3.set_title('Gráfico 3: Convergência dos Coeficientes'); ax3.set_xlabel('Passo de Tempo'); ax3.set_ylabel('Valor do Coeficiente'); ax3.legend(); ax3.grid(True)
    ax3.set_xlim(0, len(y))

    ax4.axis('off'); ax4.set_title('Estado Atual'); tabela_texto = ax4.text(0.05, 0.95, '', fontsize=9, fontfamily='monospace', va='top')

    # 4. Função que desenha cada frame da animação
    def animar(frame):
        x_n, y_n = X[frame, :], y[frame]
        previsao = filtro.predict(x_n)
        previsoes_online.append(previsao)
        historico_erro.append(y_n - previsao)
        filtro.update(x_n, y_n)
        historico_coeficientes.append(filtro.theta.copy())

        line_pred.set_data(range(len(previsoes_online)), previsoes_online)
        line_erro.set_data(range(len(historico_erro)), historico_erro)
        
        df_coeffs = pd.DataFrame(historico_coeficientes)
        for i, line in enumerate(lines_coeffs):
            line.set_data(range(len(df_coeffs)), df_coeffs.iloc[:, i])
        
        # --- AJUSTE 1: Controlar a escala do Gráfico de Erro ---
        if frame > 10:
            erro_visivel = np.array(historico_erro)
            # Foca nos 98% centrais dos dados de erro para ignorar picos extremos
            lim_inf_erro = np.percentile(erro_visivel, 1)
            lim_sup_erro = np.percentile(erro_visivel, 99)
            margem_erro = (lim_sup_erro - lim_inf_erro) * 0.1
            ax2.set_ylim(lim_inf_erro - margem_erro, lim_sup_erro + margem_erro)
            
        # --- AJUSTE 2: Controlar a escala do Gráfico de Coeficientes ---
        if frame > 10:
            df_coeffs_visivel = df_coeffs.iloc[5:] # Ignora os primeiros 5 passos
            lim_inf_coeff = df_coeffs_visivel.min().min()
            lim_sup_coeff = df_coeffs_visivel.max().max()
            margem_coeff = (lim_sup_coeff - lim_inf_coeff) * 0.1
            ax3.set_ylim(lim_inf_coeff - margem_coeff, lim_sup_coeff + margem_coeff)

        # --- AJUSTE 3: Ignorar previsões iniciais na escala do Gráfico Principal ---
        if frame > 20: # Começa a ajustar a escala do gráfico principal após 20 passos
            y_visivel = y[:frame+1]
            pred_visivel = previsoes_online[20:] # Ignora as 20 primeiras previsões para a escala
            min_val = min(np.min(y_visivel), np.min(pred_visivel))
            max_val = max(np.max(y_visivel), np.max(pred_visivel))
            margem = (max_val - min_val) * 0.1
            ax1.set_ylim(min_val - margem, max_val + margem)
        
        texto = f"Passo: {frame+1}/{len(y)}\n\nErro Atual: {historico_erro[-1]:,.2f}\n\nCoeficientes:\n"
        for i, c in enumerate(filtro.theta): texto += f" {labels_coeficientes[i]:<15} = {c:10.6f}\n"
        tabela_texto.set_text(texto)
        
        return line_pred, line_erro, *lines_coeffs, tabela_texto

    # 5. Cria e Salva a Animação
    print("Criando a animação... Isso pode levar alguns minutos.")
    ani = animation.FuncAnimation(fig, animar, frames=len(y), blit=False, interval=50) # blit=False é mais robusto para eixos dinâmicos
    nome_arquivo = f"animacao_{mac_alvo.replace(':', '')}_{server_alvo}.mp4"
    
    try:
        ani.save(nome_arquivo, writer='ffmpeg', dpi=150)
        print(f"\n✅ Animação '{nome_arquivo}' salva com sucesso!")
    except FileNotFoundError:
        print("\n❌ ERRO: `ffmpeg` não encontrado. Não foi possível salvar o vídeo.")
    except Exception as e:
        print(f"\n❌ ERRO ao salvar a animação: {e}")
    
    plt.close(fig)
    return ani

In [ ]:
# ==================================================================
# BLOCO DE EXECUÇÃO: ESCOLHA O PAR E GERE O VÍDEO
# ==================================================================
"""
Pares interessantes para análise:
'dc:a6:32:6b:9a:da', '200.159.254.239' feito e bom
'24:2f:d0:bc:6d:01', '200.159.254.239' feito e ruim
'dc:a6:32:6b:9a:da', '177.136.80.203' 
'dc:a6:32:6b:9a:da', '200.123.198.165'
'dc:a6:32:6b:9c:a8', '200.159.254.239' feito e ruim
'e4:5f:01:36:10:3e', '200.159.254.239' feito e ruim
'e4:5f:01:8e:52:7a', '177.136.80.203'
'e4:5f:01:8e:52:7a', '177.136.80.216'
'e4:5f:01:8e:52:7a', '200.159.254.239' feito e ruim
'e4:5f:01:b4:bb:d4', '200.123.198.139'
'e4:5f:01:b4:bb:d4', '200.159.254.239' feito e ruim
"""
# Escolha um dos pares que você tem nos seus dados
MAC_PARA_ANALISAR = 'e4:5f:01:b4:bb:d4' # Substitua pelo MAC que quiser
SERVER_PARA_ANALISAR = '200.123.198.139' # Substitua pelo Servidor que quiser

# Chama a função para gerar a animação para o par escolhido
# Usamos o df_limpo, que não tem os outliers
animacao_resultado = gerar_animacao_rls(df, MAC_PARA_ANALISAR, SERVER_PARA_ANALISAR)

# (Opcional) Para exibir a animação diretamente no Jupyter Notebook
#HTML(animacao_resultado.to_jshtml())